In [1]:
from datasets import load_dataset

/home/bt19d200/NER_Vamshi/NER_Model/.conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from datasets import load_dataset

dataset = load_dataset("bigbio/ddi_corpus", "ddi_corpus_bigbio_kb")

In [3]:
dataset=dataset['test']

In [4]:
dataset

Dataset({
    features: ['id', 'document_id', 'passages', 'entities', 'events', 'coreferences', 'relations'],
    num_rows: 303
})

In [5]:
dataset[0]['entities']

[{'id': '21887897_T1',
  'type': 'GROUP',
  'text': ['psychotropic drug'],
  'offsets': [[28, 45]],
  'normalized': []},
 {'id': '21887897_T2',
  'type': 'DRUG',
  'text': ['etizolam'],
  'offsets': [[90, 98]],
  'normalized': []},
 {'id': '21887897_T3',
  'type': 'DRUG',
  'text': ['phenobarbital'],
  'offsets': [[100, 113]],
  'normalized': []},
 {'id': '21887897_T4',
  'type': 'DRUG',
  'text': ['promethazine'],
  'offsets': [[115, 127]],
  'normalized': []},
 {'id': '21887897_T5',
  'type': 'DRUG',
  'text': ['chlorpromazine'],
  'offsets': [[132, 146]],
  'normalized': []},
 {'id': '21887897_T6',
  'type': 'DRUG',
  'text': ['etizolam'],
  'offsets': [[231, 239]],
  'normalized': []},
 {'id': '21887897_T7',
  'type': 'DRUG',
  'text': ['phenobarbital'],
  'offsets': [[241, 254]],
  'normalized': []},
 {'id': '21887897_T8',
  'type': 'DRUG',
  'text': ['promethazine'],
  'offsets': [[256, 268]],
  'normalized': []},
 {'id': '21887897_T9',
  'type': 'DRUG',
  'text': ['chlorpromazin

In [6]:
rel_set=set()
for data in dataset:
    for rels in data['relations']:
        rel_set.add(rels['type'])

In [7]:
rel_set

{'ADVISE', 'EFFECT', 'INT', 'MECHANISM'}

In [8]:
dataset[0]['passages']

[{'id': '21887897__text',
  'type': 'abstract',
  'text': ['An autopsy case of multiple psychotropic drug poisoning. A fatal poisoning case involving etizolam, phenobarbital, promethazine and chlorpromazine is presented. Quantitative toxicological analysis showed that the concentrations of etizolam, phenobarbital, promethazine and chlorpromazine in the femoral blood were 86 ng/ml, 5082 microg/ml, 0.107 microg/ml and 0.144 microg/ml, respectively, and large amounts of drugs were also detected in the stomach contents. We conclude that the cause of death was due to the interaction of multiple psychotropic drugs.'],
  'offsets': [[0, 556]]}]

In [9]:
dataset[0]

{'id': '21887897',
 'document_id': '21887897',
 'passages': [{'id': '21887897__text',
   'type': 'abstract',
   'text': ['An autopsy case of multiple psychotropic drug poisoning. A fatal poisoning case involving etizolam, phenobarbital, promethazine and chlorpromazine is presented. Quantitative toxicological analysis showed that the concentrations of etizolam, phenobarbital, promethazine and chlorpromazine in the femoral blood were 86 ng/ml, 5082 microg/ml, 0.107 microg/ml and 0.144 microg/ml, respectively, and large amounts of drugs were also detected in the stomach contents. We conclude that the cause of death was due to the interaction of multiple psychotropic drugs.'],
   'offsets': [[0, 556]]}],
 'entities': [{'id': '21887897_T1',
   'type': 'GROUP',
   'text': ['psychotropic drug'],
   'offsets': [[28, 45]],
   'normalized': []},
  {'id': '21887897_T2',
   'type': 'DRUG',
   'text': ['etizolam'],
   'offsets': [[90, 98]],
   'normalized': []},
  {'id': '21887897_T3',
   'type': '

In [10]:
d={}
import re
new_data=[]
for i in range(len(dataset)):
    dtokens=dataset[i]['passages'][0]['text'][0]
    dtokens = [t for t in re.split(r'[\s\-.,:;/?\]\[\(\)~>]+', dtokens) if t]

    # dtokens=re.split(r'[\s\-.,:;/?\]\[\(\)]+', dtokens)
    temp={}
    dataset[i]['tokens']=dtokens
    ner=[]
    true=[]
    for ent in dataset[i]['entities']:
        text=ent['text'][0]
        text = [t for t in re.split(r'[\s\-.,:;/?\]\[\(\)~>]+', text) if t]
        
        start=dtokens.index(text[0],0)
        end=dtokens.index(text[-1],start)
        if end-start<=3:
            ner.append([start,end,ent['text'][0],ent['type']])
        
        d[ent['id']]=ent['text'][0]

    #print(dtokens.index(text[0]),dtokens.index(text[-1]))
    for rel in dataset[i]['relations']:
        # if arg1_id not in d or arg2_id not in d:
        #     continue
        head=d[rel['arg1_id']]
        tail=d[rel['arg2_id']]
        if head==tail:
            continue
        true.append([head,tail,rel['type']])
    temp['ner']=ner
    temp['relations']=true
    temp['tokens']=dtokens
    new_data.append(temp)

        

In [11]:
new_data[0]['ner']

[[5, 6, 'psychotropic drug', 'GROUP'],
 [13, 13, 'etizolam', 'DRUG'],
 [14, 14, 'phenobarbital', 'DRUG'],
 [15, 15, 'promethazine', 'DRUG'],
 [17, 17, 'chlorpromazine', 'DRUG'],
 [13, 13, 'etizolam', 'DRUG'],
 [14, 14, 'phenobarbital', 'DRUG'],
 [15, 15, 'promethazine', 'DRUG'],
 [17, 17, 'chlorpromazine', 'DRUG']]

In [12]:
new_d={}
for i in rel_set:
    new_d[i]=0
for data in dataset:
    for rels in data['relations']:
        new_d[rels['type']]+=1

In [13]:
new_d

{'INT': 96, 'EFFECT': 360, 'MECHANISM': 302, 'ADVISE': 221}

In [14]:
len(new_data[92]['tokens'])

820

In [15]:
from pair2rel import Pair2Rel
from tqdm import tqdm
model = Pair2Rel.from_pretrained("chapalavamshi022/pair2rel")

labels=['EFFECT','MECHANISM','ADVISE','INTERACTION']

import torch

# Force usage of GPU 1
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.device = device 
relation_label_predicted_count={}
for i in tqdm(range(len(new_data))):
    if new_data[i]['ner'] and new_data[i]['relations']:
        try:
            if len(new_data[i]['tokens']) > 512:
                new_data[i]['tokens'] = new_data[i]['tokens'][:512]
            relations = model.predict_relations(new_data[i]['tokens'], labels, threshold=0.0, ner=new_data[i]['ner'], top_k=1)
        # relations = model.predict_relations(new_data[i]['sentence'], labels, threshold=0.0, ner=new_data[i]['ner'], top_k=1)
            sorted_data_desc = sorted(relations, key=lambda x: x['score'], reverse=True)
            temp=[]
            for item in sorted_data_desc:
                head=' '.join(item['head_text'])
                tail=' '.join(item['tail_text'])
                if head!=tail:
                    temp.append([head,tail,item['label']])
                    if item['label'] not in relation_label_predicted_count:
                        relation_label_predicted_count[item['label']]=0
                    relation_label_predicted_count[item['label']]+=1
                    
            new_data[i]['preds']=temp
        except:
            new_data[i]['preds']=[]
            pass
    else:
        new_data[i]['preds']=[]
        
print("Success! ✅")

/home/bt19d200/NER_Vamshi/NER_Model/.conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
100%|██████████| 303/303 [00:32<00:00,  9.27it/s]

Success! ✅


In [16]:
new_data[0]['relations']

[]

In [17]:
new_data[0]['preds']

[]

In [18]:
new_data[7]['ner']

[[0, 0, 'Ticagrelor', 'DRUG'],
 [1, 1, 'Brilinta', 'BRAND'],
 [4, 4, 'clopidogrel', 'DRUG'],
 [5, 5, 'Plavix', 'BRAND'],
 [10, 10, 'ticagrelor', 'DRUG'],
 [1, 1, 'Brilinta', 'BRAND'],
 [15, 16, 'antiplatelet drug', 'GROUP'],
 [22, 22, 'aspirin', 'BRAND'],
 [4, 4, 'clopidogrel', 'DRUG'],
 [5, 5, 'Plavix', 'BRAND'],
 [45, 45, 'prasugrel', 'DRUG'],
 [46, 46, 'Effient', 'BRAND'],
 [50, 50, 'Clopidogrel', 'DRUG']]

In [19]:
new_data[7]['relations']

[['ticagrelor', 'aspirin', 'EFFECT'], ['Brilinta', 'aspirin', 'EFFECT']]

In [20]:
new_data[7]['preds']

[['Ticagrelor', 'Clopidogrel', 'EFFECT'],
 ['Clopidogrel', 'Ticagrelor', 'MECHANISM'],
 ['ticagrelor', 'Clopidogrel', 'EFFECT'],
 ['Clopidogrel', 'aspirin', 'MECHANISM'],
 ['Clopidogrel', 'antiplatelet drug', 'MECHANISM'],
 ['Clopidogrel', 'Brilinta', 'MECHANISM'],
 ['Clopidogrel', 'ticagrelor', 'MECHANISM'],
 ['Ticagrelor', 'aspirin', 'MECHANISM'],
 ['Ticagrelor', 'clopidogrel', 'MECHANISM'],
 ['Clopidogrel', 'clopidogrel', 'MECHANISM'],
 ['Brilinta', 'Clopidogrel', 'MECHANISM'],
 ['ticagrelor', 'Ticagrelor', 'MECHANISM'],
 ['aspirin', 'Clopidogrel', 'EFFECT'],
 ['Ticagrelor', 'ticagrelor', 'MECHANISM'],
 ['Clopidogrel', 'prasugrel', 'INTERACTION'],
 ['Ticagrelor', 'antiplatelet drug', 'MECHANISM'],
 ['antiplatelet drug', 'Clopidogrel', 'MECHANISM'],
 ['clopidogrel', 'Ticagrelor', 'MECHANISM'],
 ['aspirin', 'Ticagrelor', 'MECHANISM'],
 ['Ticagrelor', 'Brilinta', 'MECHANISM'],
 ['ticagrelor', 'aspirin', 'MECHANISM'],
 ['clopidogrel', 'Clopidogrel', 'EFFECT'],
 ['Ticagrelor', 'prasugrel

In [21]:
relation_label_predicted_count

{'EFFECT': 2766, 'MECHANISM': 464, 'INTERACTION': 21969, 'ADVISE': 21}

In [22]:
new_d

{'INT': 96, 'EFFECT': 360, 'MECHANISM': 302, 'ADVISE': 221}

In [23]:
recall_avg=[]
precision_avg=[]
common=0
ours=0

for i in range(len(new_data)):
    true_set=new_data[i]['relations']
    pred_set=new_data[i]['preds']
    d={}
    for item in pred_set:
        if (item[0],item[1]) not in d and (item[1],item[0]) not in d:
            d[(item[0],item[1])]=item[2]
    
    ours+=len(d)
    if len(true_set) == 0 and len(pred_set) == 0:
        recall_avg.append(1.0)
        precision_avg.append(1.0)
    else:
        count=0
        for m in true_set:
            for n in pred_set:
                if (m[0] in n[0]) and (m[1] in n[1]) and (m[2]==n[2] or (m[2]=='INT' and n[2]=='INTERACTION')):
                    count+= 1
        recall = count/ len(true_set) if len(true_set) > 0 else 0
        precision = count / len(d) if len(d) > 0 else 0
        common += count
        recall_avg.append(recall)
        precision_avg.append(precision)

In [24]:
sum(recall_avg)/len(recall_avg),sum(precision_avg)/len(precision_avg)

(0.4824456153031852, 0.41832432770981515)

In [25]:
from pair2rel import Pair2Rel
from tqdm import tqdm
model = Pair2Rel.from_pretrained("chapalavamshi022/pair2rel")

labels=['EFFECT','MECHANISM','ADVISE','INTERACTION']

import torch

# Force usage of GPU 1
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.device = device 
relation_label_predicted_count={}
for i in tqdm(range(len(new_data))):
    if new_data[i]['ner'] and new_data[i]['relations']:
        try:
            if len(new_data[i]['tokens']) > 512:
                new_data[i]['tokens'] = new_data[i]['tokens'][:512]
            relations = model.predict_relations(new_data[i]['tokens'], labels, threshold=0.0, ner=new_data[i]['ner'], top_k=2)
        # relations = model.predict_relations(new_data[i]['sentence'], labels, threshold=0.0, ner=new_data[i]['ner'], top_k=1)
            sorted_data_desc = sorted(relations, key=lambda x: x['score'], reverse=True)
            temp=[]
            for item in sorted_data_desc:
                head=' '.join(item['head_text'])
                tail=' '.join(item['tail_text'])
                if head!=tail:
                    temp.append([head,tail,item['label']])
                    if item['label'] not in relation_label_predicted_count:
                        relation_label_predicted_count[item['label']]=0
                    relation_label_predicted_count[item['label']]+=1
                    
            new_data[i]['preds']=temp
        except:
            new_data[i]['preds']=[]
            pass
    else:
        new_data[i]['preds']=[]
        
print("Success! ✅")

100%|██████████| 303/303 [00:31<00:00,  9.58it/s]

Success! ✅


In [26]:
recall_avg=[]
precision_avg=[]
common=0
ours=0

for i in range(len(new_data)):
    true_set=new_data[i]['relations']
    pred_set=new_data[i]['preds']
    d={}
    for item in pred_set:
        if (item[0],item[1]) not in d and (item[1],item[0]) not in d:
            d[(item[0],item[1])]=item[2]
    
    ours+=len(d)
    if len(true_set) == 0 and len(pred_set) == 0:
        recall_avg.append(1.0)
        precision_avg.append(1.0)
    else:
        count=0
        for m in true_set:
            for n in pred_set:
                if (m[0]==n[0]) and (m[1]==n[1]) and (m[2]==n[2] or (m[2]=='INT' and n[2]=='INTERACTION')):
                    count+= 1
        recall = count/ len(true_set) if len(true_set) > 0 else 0
        precision = count / len(d) if len(d) > 0 else 0
        common += count
        recall_avg.append(recall)
        precision_avg.append(precision)

In [27]:
sum(recall_avg)/len(recall_avg),sum(precision_avg)/len(precision_avg)

(0.5737120386899685, 0.45682144400673336)

In [28]:
from pair2rel import Pair2Rel
from tqdm import tqdm
model = Pair2Rel.from_pretrained("chapalavamshi022/pair2rel")

labels=['EEFECT','MECHANISM','ADVISE','INT']

import torch

# Force usage of GPU 1
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.device = device 
relation_label_predicted_count={}
for i in tqdm(range(len(new_data))):
    if new_data[i]['ner'] and new_data[i]['relations']:
        try:
            if len(new_data[i]['tokens']) > 512:
                new_data[i]['tokens'] = new_data[i]['tokens'][:512]
            relations = model.predict_relations(new_data[i]['tokens'], labels, threshold=0.0, ner=new_data[i]['ner'], top_k=3)
        # relations = model.predict_relations(new_data[i]['sentence'], labels, threshold=0.0, ner=new_data[i]['ner'], top_k=1)
            sorted_data_desc = sorted(relations, key=lambda x: x['score'], reverse=True)
            temp=[]
            for item in sorted_data_desc:
                head=' '.join(item['head_text'])
                tail=' '.join(item['tail_text'])
                if head!=tail:
                    temp.append([head,tail,item['label']])
                    if item['label'] not in relation_label_predicted_count:
                        relation_label_predicted_count[item['label']]=0
                    relation_label_predicted_count[item['label']]+=1
                    
            new_data[i]['preds']=temp
        except:
            new_data[i]['preds']=[]
            pass
    else:
        new_data[i]['preds']=[]
        
print("Success! ✅")

100%|██████████| 303/303 [00:32<00:00,  9.41it/s]

Success! ✅


In [29]:
sum(recall_avg)/len(recall_avg),sum(precision_avg)/len(precision_avg)

(0.5737120386899685, 0.45682144400673336)